<a href="https://colab.research.google.com/github/arya2910/Text-to-Text-generation/blob/main/texttotext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("/content/jokes.csv")
df

In [ ]:
import re
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Convert list of strings to single string
text = ' '.join([str(x) for x in df['Question']])

# Define a function to clean the text
def clean_text(text):
    # Remove unwanted characters and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text).lower()
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if not word in stop_words]
    # Join tokens into a string
    text = ' '.join(tokens)
    return text

In [ ]:
cleaned_data_x = []
for i in range(len(df['Question'])):
    cleaned_data_x.append(clean_text(df['Question'][i]))

In [ ]:
cleaned_data_y = []
for i in range(len(df['Answer'])):
    cleaned_data_y.append(clean_text(df['Answer'][i]))

In [ ]:
print(len(cleaned_data_x))
print(len(cleaned_data_y))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import one_hot
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed,Activation, concatenate
from keras.callbacks import EarlyStopping
from keras.layers import dot
import numpy
from keras import backend as K

In [ ]:
max_input_length = 50
max_output_length = 50

tokenizer_x = Tokenizer()
tokenizer_x.fit_on_texts(cleaned_data_x)
input_sequences = tokenizer_x.texts_to_sequences(cleaned_data_x)
print(input_sequences)
encoder_inputs = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
print(encoder_inputs[0])
input_vocab_size = len(tokenizer_x.word_index) + 1
print(len(tokenizer_x.word_index))

In [ ]:
tokenizer_y = Tokenizer()
tokenizer_y.fit_on_texts(cleaned_data_y)
output_sequences = tokenizer_y.texts_to_sequences(cleaned_data_y)
decoder_inputs = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')
output_vocab_size= len(tokenizer_y.word_index) + 1

In [ ]:
print(len(encoder_inputs))
print(len(decoder_inputs))

In [ ]:
embedding_dim = 128
latent_dim = 256

encoder_inputs = Input(shape=(max_input_length,))
print(encoder_inputs)
encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(max_output_length,))
decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

In [ ]:
attention = dot([decoder_outputs, encoder_outputs], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)

context = dot([attention, encoder_outputs], axes=[2, 1])
decoder_combined_context = concatenate([context, decoder_outputs])

decoder_dense = TimeDistributed(Dense(output_vocab_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_combined_context)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Train the model
batch_size = 64
epochs = 100
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()

In [ ]:
import numpy as np
import tensorflow as tf

encoder_inputs_np = np.array(encoder_inputs)
decoder_inputs_np = np.array(decoder_inputs)
decoder_outputs_np = np.array(decoder_outputs)

model.fit([tf.convert_to_tensor(encoder_inputs_np), tf.convert_to_tensor(decoder_inputs_np)], 
          tf.convert_to_tensor(decoder_outputs_np),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          callbacks=[early_stopping])


In [ ]:
model.fit([encoder_inputs, decoder_inputs], decoder_outputs,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          callbacks=[early_stopping])



```
# This is formatted as code
sentences = tokenized_data # assuming clean_text is defined beforehand
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)
padded_sequences = pad_sequences(sequences, maxlen=70, padding='post')
print(len(padded_sequences))
```

